IMPORTANT: First install bs4 from the terminal if there is no previous : pip install beautifulsoup4


In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


There are couples of options when you are scraping and parsing html, you can parse it into string, or you can parse it into html files

### Import data as dataframe

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
#print(soup.prettify())

match = soup.table.text

table will get the tag element together with text
If you use match method, it will match the first tag, find would be better and more specificdiv

match = soup.find('div',class ='_footer_')
match = soup.find('div', id ='footer')

In this assignment, we use find(). 

first we use find() to find the content of table and store it in one variable called table, then we use find_all() to find all content of attributes of 'td' 

In [3]:
table = soup.find('table')
fields = table.find_all('td')
print(fields[0:20])
print(len(fields))

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
</td>, <td>M2A</td>, <td>Not assigned</td>, <td>Not assigned
</td>, <td>M3A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td>, <td>M4A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>, <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td>, <td>M5A</td>, <td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>, <td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td>, <td>M6A</td>, <td><a href="/wiki/North_York" title="North York">North York</a></td>]
864


Some explaination of syntax:
range() function syntax range (start, stop[, step])

string.strip()

Example:
str = "0000000this is string example....wow!!!0000000";
print str.strip( '0' )

output: this is string example....wow!!!

The following code is going to create 3 lists to store Postcode, Borough and Neighbourhood respectively. And append the content in every td tag

In [8]:
postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        

The following codes will append 3 list into a dataframe and we will transpose it


In [9]:
df1 = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df1.columns = ['Postcode', 'Borough', 'Neighbourhood']
df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only keep assigned column and drop unassigned col

In [10]:
df1['Borough'].replace('Not assigned', np.nan, inplace=True)
df1.dropna(subset=['Borough'], inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combine the borough with the same postcode.
The combination of Postcode and Borough should be unique after groupby.
Combine the values of Neighbourhood by and use ',' as delimilator


In [11]:
df2 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2.columns = ['Postcode', 'Borough', 'Neighbourhood']
df2.shape
df2.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
df2.shape

(103, 3)

#### Read geo data as a dataframe, give it column names 


In [14]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.columns = ['Postcode', 'Latitude', 'Longitude']
geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### merge this dataframe with df2 on column Postcode. 

In [16]:
dfFinal = pd.merge(df2, geo, on=['Postcode'], how='inner')
dfFinal.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
